# TASK ON REAL TIME TRANSLATION OF AUDIO FROM ENGLISH TO HINDI(OR ANY OTHER CHOOSEN LANGUAGE)

# CAUTION!!!

### INORDER TO USE THE LIBRARY PYUDUB , we require python versions below 3.13

In [ ]:
%pip install sounddevice scipy numpy
%pip install requests pandas pydub pyaudio 
%pip install simpleaudio
%pip install dotenv soundfile
%pip install playsound

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not wr

In [28]:
import wave
import base64
import requests
import sounddevice as sd
import numpy as np
import time
import os
from pydub import AudioSegment

In [29]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("API_KEY")

# RECORD THE AUDIO MODULE

In [ ]:
import sounddevice as sd
import soundfile as sf
import threading

def record_audio(output_file='input_audio.wav', fs=44100):
    """Continuously record audio until user presses Enter, then save to a WAV file."""
    print("Recording... Press Enter to stop.")

    recording = []
    stop_recording = threading.Event()

    def wait_for_input():
        input()
        stop_recording.set()

    threading.Thread(target=wait_for_input).start()

    with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
        while not stop_recording.is_set():
            audio_chunk, _ = stream.read(1024)
            recording.append(audio_chunk)

    print("Recording stopped.")

    audio_data = np.concatenate(recording, axis=0)
    with wave.open(output_file, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(fs)
        wf.writeframes(audio_data)

    print(f"Audio saved to {output_file}")
    return output_file



# SPEECH TO TEXT TRANSLATION MODULE

In [51]:
def speech_to_text(file_path,code):
    
    api_url = "https://api.sarvam.ai/speech-to-text"
    
    headers = {
        "api-subscription-key": api_key
        }
    
    data = {
        "language_code": code,  
        "model": "saarika:v2",     
        "with_timestamps": False   
    }

    with open(file_path, 'rb') as audio_file:
        files = {'file': ('audio.wav', audio_file, 'audio/wav')}
        response = requests.post(api_url, headers=headers, files=files, data=data)

    if response.status_code == 200 or response.status_code == 201:
        response_data = response.json()
        transcript = response_data.get("transcript")
        return transcript
    else:
        print(f"ERROR: {response.status_code}")
        print("Response:", response.text)
        return None


# TEXT TO SPEECH MODULE

In [52]:
def text_to_speech(transcript,code):
    api_url = "https://api.sarvam.ai/text-to-speech"
    
    headers = {
        "Content-Type": "application/json",
        "api-subscription-key": api_key
    }
    
    payload = {
        "inputs": [transcript],
        "target_language_code": code,
        "speaker": "abhilash",
        "model": "bulbul:v2",
        "pitch": 0,
        "pace": 1.0,
        "loudness": 1.0,
        "enable_preprocessing": True,
    }
    
    response = requests.post(api_url, json=payload, headers=headers)
    if response.status_code == 200:
        audio = response.json()["audios"][0]
        audio = base64.b64decode(audio)

        output_file = "output.wav"
        with wave.open(output_file, "wb") as wav_file:
            wav_file.setnchannels(1)  
            wav_file.setsampwidth(2)  
            wav_file.setframerate(22050)  
            wav_file.writeframes(audio)

        if os.path.exists(output_file):
            print(f"Audio file saved successfully as {output_file}")
        else:
            print(f"Failed to save audio file.")
    else:
        print(f"Error: {response.status_code}")
        print(response.json())

# MODULE TO PLAY THE AUDIO BACK

In [53]:
import threading

def play_audio(file_path):
    import soundfile as sf
    import sounddevice as sd
    data, fs = sf.read(file_path, dtype='float32')
    sd.play(data, fs)
    sd.wait()


### TO USE THE PIPELINE RUN THE CELL AND RECORD THE AUDIO, TO STOP THE RECORDING CLICK ON ENTER AGAIN. THE TRANSLATED AUDIO WILL START TO PLAY

In [57]:
code="hi-IN"


record_audio("input.wav")
transcript = speech_to_text("input.wav",code)
print(f"Transcript: {transcript}")
text_to_speech(transcript,code)
play_audio("output.wav")


Recording... Press Enter to stop.
Recording stopped.
Audio saved to input.wav
Transcript: अगर वह एक लज़ी बोन होता है, तो मैं उसके भविष्य के अरुरूप से डरहूँगा हूँ और वह मेरी अनुरोधकों को समझ नहीं समझता। मैं उसके भविष्य के बारे में सूरज हूँ।
Audio file saved successfully as output.wav
